# Práctico 10: Cognición numérica

En este práctico intentaremos replicar los resultados reportados por Nasr y colaboradores (2019) usando una red convolucional profunda de menor tamaño reportada por Kubilius y cols. (2018) llamada _CORnet-Z_. La arquitectura de esta red intenta alinearse con la actividad neuronal de los humanos, de hecho, esta separada en áreas V1, V2, V4 e IT. Los autores del primer articulo utilizaron una red convolucional profunda más grande, pero también más dificil de entrenar, es por esto que optamos por utilizar una red más chica para comparar.

## Configuración

Ejecutá todas las celdas de esta sección para importar las librerías y funciones que vamos a utilizar en el práctico.

In [ ]:
import collections, math, requests
import torch
import torch.nn as nn
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import ipywidgets as widgets

from PIL import Image, ImageDraw

### Funciones utilitarias

In [ ]:
def encontrar_device():
  if torch.cuda.is_available():
    device = torch.device("cuda")
  elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = torch.device("mps")
  else:
    device = torch.device("cpu")
  print("Device encontrado:", device)
  return device

def descargar_clases_imagenet()
    url = 'https://raw.githubusercontent.com/MaestriaCienciasCognitivas/ncc/main/book/static/Practico10_Imagenet.json'
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    clases = [v[1] for k, v in sorted(data.items(), key=lambda kv: int(kv[0]))]
    return clases

def muestrear_estandar(tam_imagen, num_objetos, radio_nominal, ruido_radio=0.1, distancia_min=5, margen=5, max_iter=10_000):
    x, y, r = None, None, None
    for i in range(num_objetos):
        for j in range(max_iter):
            ri = np.round(radio_nominal + ruido_radio * radio_nominal * np.random.normal()).astype(int)
            xi = np.round(np.random.uniform(ri + margen, tam_imagen - ri - margen)).astype(int)
            yi = np.round(np.random.uniform(ri + margen, tam_imagen - ri - margen)).astype(int)
            if i == 0:
                x, y, r = xi, yi, ri
                break
            else:
                dists = np.sqrt((x - xi) ** 2 + (y - yi) ** 2)
                if np.all(dists > r + ri + distancia_min):
                    x = np.append(x, xi)
                    y = np.append(y, yi)
                    r = np.append(r, ri)
                    break
                if j == max_iter - 1:
                    return None
    return x, y, r

def muestrear_area_constante(tam_imagen, num_objetos, area_total, **kwargs):
    radio_nominal = np.sqrt(area_total / num_objetos / np.pi)
    return muestrear_estandar(tam_imagen, num_objetos, radio_nominal, **kwargs)

def muestrear_casco_convexo(tam_imagen, num_objetos, radio_nominal, ruido_radio=0.1, margen=5, distancia_min=5, max_iter=50_000, radio_casco=85, tam_hull=5):
    x, y, r = None, None, None
    cx = cy = tam_imagen / 2
    theta_hull = np.arange(0, 2 * np.pi, (2 * np.pi) / tam_hull)
    theta_hull += np.random.uniform(high=np.pi)
    np.random.shuffle(theta_hull)
    for i in range(num_objetos):
        for j in range(max_iter):
            if i < tam_hull:
                theta = theta_hull[i]
                radio = radio_casco + 5 * np.random.normal()
            else:
                theta = np.random.uniform(high=2 * np.pi)
                radio = np.random.uniform(high=radio_casco - 2 * radio_nominal)
            xi = np.round(cx + radio * np.cos(theta)).astype(int)
            yi = np.round(cy + radio * np.sin(theta)).astype(int)
            ri = np.round(radio_nominal + ruido_radio * radio_nominal * np.random.normal()).astype(int)
            if i == 0:
                x, y, r = xi, yi, ri
                break
            else:
                dists = np.sqrt((x - xi) ** 2 + (y - yi) ** 2)
                if np.all(dists > r + ri + distancia_min):
                    x = np.append(x, xi)
                    y = np.append(y, yi)
                    r = np.append(r, ri)
                    break
                if j == max_iter - 1:
                    return None
    return x, y, r

def verificar_densidad_control(x, y, r, dist_min=90, dist_max=100):
    coords = np.concatenate((x[:, None], y[:, None]), axis=1)
    dists = sp.spatial.distance.cdist(coords, coords, 'euclidean')
    avg_dist = dists[np.triu_indices(len(x), 1)].mean()
    return (avg_dist >= dist_min) & (avg_dist <= dist_max)

def generar_fondo_uniforme(size, A):
    h, w, ch = size
    img = Image.new("L", (w, h), color=A)
    arr = np.array(img, dtype=np.uint8)
    if ch == 1:
        arr = arr[..., None]
    elif ch == 3:
        arr = np.repeat(arr[..., None], 3, axis=2)
    return arr

def dibujar_circulo(img, x, y, r):
    if img.ndim == 3 and img.shape[2] == 1:
        base = img[..., 0]
    else:
        base = img
    pil_img = Image.fromarray(base)
    draw = ImageDraw.Draw(pil_img)
    bbox = [x - r, y - r, x + r, y + r]
    draw.ellipse(bbox, fill=255)
    out = np.array(pil_img, dtype=np.uint8)
    if img.ndim == 3 and img.shape[2] == 1:
        out = out[..., None]
    return out

def dibujar_forma_aleatoria(img, x, y, r):
    if img.ndim == 3 and img.shape[2] == 1:
        base = img[..., 0]
    else:
        base = img
    pil_img = Image.fromarray(base)
    draw = ImageDraw.Draw(pil_img)
    ss = np.random.choice(range(4))
    if ss == 0:  # círculo
        bbox = [x - r, y - r, x + r, y + r]
        draw.ellipse(bbox, fill=255)
    elif ss == 1:  # rectángulo
        r1 = int(np.random.uniform(0.7, 1.0) * r)
        r2 = int(np.random.uniform(0.7, 1.0) * r)
        draw.rectangle([x - r1, y - r2, x + r1, y + r2], fill=255)
    elif ss == 2:  # elipse
        r1 = int(np.random.uniform(0.3, 1.0) * r)
        r2 = int(np.random.uniform(0.3, 1.0) * r)
        bbox = [x - r1, y - r2, x + r1, y + r2]
        draw.ellipse(bbox, fill=255)
    elif ss == 3:  # triángulo
        r1 = int(np.random.uniform(0.7, 1.0) * r)
        r2 = int(np.random.uniform(0.7, 1.0) * r)
        r3 = int(np.random.uniform(0.7, 1.0) * r)
        pts = [(x + r1, y), (x, y - r2), (x - r3, y)]
        draw.polygon(pts, fill=255)
    out = np.array(pil_img, dtype=np.uint8)
    if img.ndim == 3 and img.shape[2] == 1:
        out = out[..., None]
    return out

def generar_conjunto(numerosidades, repeticiones, fn_muestreo, args_muestreo, fn_verificacion, fn_dibujo, tam_imagen, max_iter, nivel_fondo=50):
    S, Q = [], []
    for n in numerosidades:
        for _ in range(repeticiones):
            img = generar_fondo_uniforme((tam_imagen, tam_imagen, 1), A=nivel_fondo)
            if n > 0:
                for v in range(max_iter):
                    x, y, r = fn_muestreo(tam_imagen, n, **args_muestreo)
                    if n > 1:
                        if fn_verificacion(x, y, r):
                            break
                    else:
                        break
                    if v == max_iter - 1:
                        return None, None
                if n == 1:
                    x, y, r = [x], [y], [r]
                for xi, yi, ri in zip(x, y, r):
                    img = fn_dibujo(img, xi, yi, ri)
            S.append(img)
            Q.append(n)
    S = np.array(S)
    Q = np.array(Q)
    randperm = np.random.permutation(len(Q))
    return S[randperm], Q[randperm]

def generar_estimulos(num_reps=40, rango_Q=np.array([0, 1, 2, 3, 4]), radio_punto=18, area_total=1200, tam_hull=3, tam_imagen=224):
    Ss, Qs = generar_conjunto(
        numerosidades=rango_Q,
        repeticiones=num_reps,
        fn_muestreo=muestrear_estandar,
        args_muestreo={'radio_nominal': radio_punto},
        fn_verificacion=lambda x, y, r: True,
        fn_dibujo=dibujar_circulo,
        tam_imagen=tam_imagen,
        max_iter=1000,
        nivel_fondo=50
    )
    Sc, Qc = generar_conjunto(
        numerosidades=rango_Q,
        repeticiones=num_reps,
        fn_muestreo=muestrear_area_constante,
        args_muestreo={'area_total': area_total},
        fn_verificacion=verificar_densidad_control,
        fn_dibujo=dibujar_circulo,
        tam_imagen=tam_imagen,
        max_iter=10_000,
        nivel_fondo=50
    )
    mean_por_imagen = Sc.reshape((Sc.shape[0], -1)).mean(axis=1)
    mean_fija = mean_por_imagen.min()
    Sc = mean_fija * (Sc / mean_por_imagen[:, None, None, None])
    Sss, Qss = generar_conjunto(
        numerosidades=rango_Q,
        repeticiones=num_reps,
        fn_muestreo=muestrear_casco_convexo,
        args_muestreo={'radio_nominal': radio_punto, 'tam_hull': tam_hull},
        fn_verificacion=lambda x, y, r: True,
        fn_dibujo=dibujar_forma_aleatoria,
        tam_imagen=tam_imagen,
        max_iter=1000,
        nivel_fondo=50
    )
    S = np.concatenate((Ss, Sc, Sss))
    Q = np.concatenate((Qs, Qc, Qss))
    C = np.concatenate((
        0 * np.ones_like(Qs),
        1 * np.ones_like(Qc),
        2 * np.ones_like(Qss),
    ))
    S = np.tile(S, (1, 1, 1, 3))
    S = S.transpose((0, 3, 1, 2))
    randperm = np.random.permutation(len(Q))
    S, Q, C = S[randperm], Q[randperm], C[randperm]
    S = S.astype(np.float32) / 255.0
    Q = Q.astype(int)
    C = C.astype(int)
    return S, Q, C

def anova_two_way(A, B, Y):
    num_cells = Y.shape[1]

    A_levels = np.unique(A); a = len(A_levels)
    B_levels = np.unique(B); b = len(B_levels)
    Y4D = np.array([[Y[(A==i)&(B==j)] for j in B_levels] for i in A_levels])

    r = Y4D.shape[2]

    Y = Y4D.reshape((-1, Y.shape[1]))

    # only test cells (units) that are active (gave a nonzero response to at least one stimulus) to avoid division by zero errors
    active_cells = np.where(np.abs(Y).max(axis=0)>0)[0]
    Y4D = Y4D[:,:,:,active_cells]
    Y = Y[:, active_cells]

    N = Y.shape[0]

    Y_mean = Y.mean(axis=0)
    Y_mean_A = Y4D.mean(axis=1).mean(axis=1)
    Y_mean_B = Y4D.mean(axis=0).mean(axis=1)
    Y_mean_AB = Y4D.mean(axis=2)


    SSA = r*b*np.sum((Y_mean_A - Y_mean)**2, axis=0)
    SSB = r*a*np.sum((Y_mean_B - Y_mean)**2, axis=0)
    SSAB = r*((Y_mean_AB - Y_mean_A[:,None] - Y_mean_B[None,:] + Y_mean)**2).sum(axis=0).sum(axis=0)
    SSE = ((Y4D-Y_mean_AB[:,:,None])**2).sum(axis=0).sum(axis=0).sum(axis=0)
    SST = ((Y-Y_mean)**2).sum(axis=0)

    DFA = a - 1; DFB = b - 1; DFAB = DFA*DFB
    DFE = (N-a*b); DFT = N-1

    MSA = SSA / DFA
    MSB = SSB / DFB
    MSAB = SSAB / DFAB
    MSE = SSE / DFE

    FA = MSA / MSE
    FB = MSB / MSE
    FAB = MSAB / MSE

    pA = np.nan*np.zeros(num_cells)
    pB = np.nan*np.zeros(num_cells)
    pAB = np.nan*np.zeros(num_cells)

    pA[active_cells] = sp.stats.f.sf(FA, DFA, DFE)
    pB[active_cells] = sp.stats.f.sf(FB, DFB, DFE)
    pAB[active_cells] = sp.stats.f.sf(FAB, DFAB, DFE)

    return pA, pB, pAB

def average_tuning_curves(Q, H, rango_Q):
    Qrange = np.unique(Q)
    tuning_curves = np.array([H[Q==j,:].mean(axis=0) for j in rango_Q])
    return tuning_curves

def preferred_numerosity(Q, H, rango_Q):
    tuning_curves = average_tuning_curves(Q, H, rango_Q)
    pref_num = np.unique(Q)[np.argmax(tuning_curves, axis=0)]
    return pref_num

### Funciones de graficado

In [ ]:
def visualizar_activaciones(activationes, capa):
    act = activationes[capa][0]
    n = act.shape[0]
    
    cols = 12
    rows = int(math.ceil(n / cols))
    
    fig, axes = plt.subplots(rows, cols, figsize=(12, 1.2*rows))
    axes = axes.flatten()
    
    for i in range(n):
      axes[i].imshow(act[i], cmap="gray")
      axes[i].axis("off")
    
    for j in range(n, len(axes)):
      axes[j].axis("off")
    
    plt.suptitle(f"Activaciones en {capa} ({n} canales de {act.shape[1]}x{act.shape[2]})")
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

def visualizar_pesos(conv_layer, in_channel=0, max_plots=64):
    W = conv_layer.weight.detach().cpu()
    n = min(W.shape[0], max_plots)
    cols = int(math.ceil(math.sqrt(n)))
    rows = int(math.ceil(n / cols))
    
    fig, axes = plt.subplots(rows, cols, figsize=(2*cols, 2*rows))
    axes = axes.flatten()
    
    for i in range(n):
        ker = W[i, in_channel].numpy()
        axes[i].imshow(ker, cmap="bwr")
        axes[i].axis("off")
    for j in range(n, len(axes)):
        axes[j].axis("off")
    plt.suptitle(f"Pesos {conv_layer.__class__.__name__}, canal entrada {in_channel}")
    plt.show()

### CORnet-Z

In [ ]:
class Flatten(nn.Module):
  """
  Helper module for flattening input tensor to 1-D for the use in Linear modules
  """
  def forward(self, x):
    return x.view(x.size(0), -1)

class Identity(nn.Module):
  """
  Helper module that stores the current tensor. Useful for accessing by name
  """
  def forward(self, x):
    return x

class CORblock_Z(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
    super().__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                       stride=stride, padding=kernel_size // 2)
    self.nonlin = nn.ReLU(inplace=True)
    self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    self.output = Identity()  # for an easy access to this block's output

  def forward(self, inp):
    x = self.conv(inp)
    x = self.nonlin(x)
    x = self.pool(x)
    x = self.output(x)  # for an easy access to this block's output
    return x


def CORnet_Z(device = torch.device('cpu')):
  model = nn.Sequential(collections.OrderedDict([
    ('V1', CORblock_Z(3, 64, kernel_size=7, stride=2)),
    ('V2', CORblock_Z(64, 128)),
    ('V4', CORblock_Z(128, 256)),
    ('IT', CORblock_Z(256, 512)),
    ('decoder', nn.Sequential(collections.OrderedDict([
      ('avgpool', nn.AdaptiveAvgPool2d(1)),
      ('flatten', Flatten()),
      ('linear', nn.Linear(512, 1000)),
      ('output', Identity())
    ])))
  ]))

  model = nn.DataParallel(model)

  url = 'https://s3.amazonaws.com/cornet-models/cornet_z-5c427c9c.pth'
  ckpt_data = torch.hub.load_state_dict_from_url(url, map_location=device)
  model.load_state_dict(ckpt_data['state_dict'])

  return model.to(device)

## Instanciación de la red

Instanciá la red de CORnet-Z y fijate en su arquitectura. ¿Te suenan algunos componentes de los practicos de convolución?

CORnet-Z fue entrando en una tarea de clasificación. En particular, de 1000 categorías del dataset Imagenet. Es por eso que en la capa de _decoder_ ves una capa lineal de 1000 unidades. Igual, no importa, en nuestros análisis no vamos a fijarnos en esta capa.

In [ ]:
device = encontrar_device()
cnn = CORnet_Z(device)
print(cnn.module)

Para explorar CORnet-Z, en lugar de _Imagenet_, vamos a usar _Imagenette_, un subjconjunto (sus imagenes y categorías también estan en _Imagenet_) muy reducido que nos permite por manos a la obra mucho mas rápido. Usa el siguiente componente para explorar algunas imagenes y sus categorías. CORnet-Z fue entrenado en un número de categorias 10 veces mayor.

In [ ]:
from torchvision.datasets import Imagenette
from torchvision.transforms import Compose, Resize, ToTensor

transform = Compose([Resize((224, 224)), ToTensor()])
dataset = Imagenette(root='data', size="160px", transform=transform, download=False)

@widgets.interact(idx=(0, len(dataset)-1))
def visualizar_imagen(idx):
    imagen, clase = dataset[idx]
    plt.imshow(imagen.squeeze().numpy().transpose((1, 2, 0)))
    plt.show()
    print(f'Clase: {dataset.classes[clase][1]}')

¿Que tal le va a CORnet-Z clasificando estas imágenes? Explorá alguna de ellas y fijate las probabilidades que arroja la inferencia.

In [ ]:
# Paso de normalización para usar las imagenes en CORnet-z
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# Descargamos la lista con las clases de Imagenet
clases = descargar_clases_imagenet()

@widgets.interact(idx=(0, len(dataset)-1))
def visualizar_imagen(idx):
    imagen, _ = dataset[idx]
    plt.imshow(imagen.squeeze().numpy().transpose((1, 2, 0)))
    plt.show()
    
    imagen = normalize(imagen).unsqueeze(0).to(device)
    logits = cnn(imagen)    
    
    probs = torch.softmax(logits, dim=1)
    top_probs, top_idxs = torch.topk(probs, k=5, dim=1)
    for p, idx_cls in zip(top_probs[0], top_idxs[0]):
        clase = clases[idx_cls.item()]
        print(f"{clase:25s}  probabilidad = {p.item():.4f}")

## Instanciación de los estímulos

Vayamos al artículo de Nasr y colaboradores. ¿Qué querían investigar? Si no te queda claro, es un buen momento para discutirlo en clase.

Los autores del artículo generaron estímulos en forma controlada, nosotros haremos lo mismo. Ejecutá la celda siguiente para generarlos.

In [ ]:
# Numpy array con las numerosidades
rango_Q = np.array([0, 1, 2, 3, 4])

# Semilla para replicar los resultados
np.random.seed(12345)

# Generación de los estímulos
S, Q, C = generar_estimulos(num_reps=40, rango_Q=rango_Q, radio_punto=18, tam_hull=3)

print("Forma de S:", S.shape)
print("Forma de Q:", Q.shape)
print("Forma de C:", C.shape)

La matriz `S` contiene 600 imagenes a color (RGB: un canal para el rojo, otro para el verde y el tercero para el azul) de tamaño $224 \times 224$. Usando el siguiente componente, investigá que información comunican `Q` y `C`.

In [ ]:
@widgets.interact(i=(0, len(S)-1))
def visualizar_dataset(i):
    fig, ax = plt.subplots(1, 1, figsize=(2, 2))
    ax.set_axis_off()
    ax.imshow(S[i].transpose((1, 2, 0)))
    plt.title(f"Q: {Q[i]}, C: {C[i]}")
    plt.show()
    plt.close(fig)

¿Cómo se activarán las unidades de las distintas capas de CORnet-Z al "ver" estas imágenes de estímulo que generaste en forma controlada? Usa el componente siguiente para explorarlo.

In [ ]:
activations = {}

def make_hook(name):
    def _hook(m, i, o): activations[name] = o.detach().cpu()
    return _hook

hooks = [
    cnn.module.V1.register_forward_hook(make_hook('V1')),
    cnn.module.V2.register_forward_hook(make_hook('V2')),
    cnn.module.V4.register_forward_hook(make_hook('V4')),
    cnn.module.IT.register_forward_hook(make_hook('IT'))
]

@widgets.interact(i=(0, len(S)-1), capa=['V1', 'V2', 'V4', 'IT'])
def simular_inferencia(i, capa):
    activations.clear()
    s = torch.tensor(S[i]).unsqueeze(0).to(device)
    out = cnn(s)
    visualizar_activaciones(activations, capa)

Ahora, lo que vamos a hacer es efectuar inferencias para las 600 imágenes al mismo tiempo, y quedarnos con todas las activaciones de la capa IT.

Al imprimir la forma de la matriz, se puede apreciar que tiene un total de 25088 unidades. ¿Cómo se llega a este número?

In [ ]:
activations.clear()
s = torch.tensor(S).to(device)
out = cnn(s)
IT = activations["IT"].flatten(1).numpy()
print(IT.shape)

A continuación, hacemos un análisis de ANOVA de dos vías para detectar aquellas unidades cuyas activaciones estan moduladas por Q (la numerosidad) independientemente de la C (la condición).

In [ ]:
pN, pC, pNC = anova_two_way(Q, C, IT)
anova_cells = np.where((pN<0.01) & (pNC>0.01) & (pC>0.01))[0]
print('Número de unidades con numerosidad preferida = %i (%0.2f%%)'%(len(anova_cells), 100*len(anova_cells)/IT.shape[1]))

H = IT[:, anova_cells]
print("Forma de H:", H.shape)

Con la función `preferred_numerosity` podemos calcular cual es la numerosidad preferida para cada unidad. Usá el componente interactivo para entender qué significa que una unidad pueda tener una preferencia.

In [ ]:
pref_num = preferred_numerosity(Q, H, rango_Q)

@widgets.interact(unit=(0, len(H)))
def graficar_curva(unit):
    plt.figure(figsize=(8,4))
    tc = np.array([H[(Q==q) & (C==j), unit].mean() for q in rango_Q])
    plt.plot(rango_Q, tc, linewidth=0.5)

    tc = np.array([H[(Q==q), unit].mean() for q in rango_Q])
    err = np.array([H[(Q==q), unit].std() for q in rango_Q]) / np.sqrt(np.sum((Q==Q[0])))
    plt.errorbar(rango_Q, tc, err, color='r', linewidth=1.5)

    plt.xlabel('Numerosidad'); plt.ylabel('Activación')
    plt.title(f'Unidad {unit} (numerosidad preferida = {pref_num[unit]})')
    plt.xticks(rango_Q)
    plt.tight_layout()
    plt.show()

La siguiente gráfica muestra cuantas unidades prefieren cada número. ¿Coincide con lo reportado por los autores?

In [ ]:
hist = [np.sum(pref_num==q) for q in rango_Q]
hist /= np.sum(hist)

plt.figure(figsize=(4,4))
plt.bar(rango_Q, 100*hist, width=0.8)
plt.xlabel('Numerosidad preferida')
plt.ylabel('Porcentaje de unidades')
plt.show()

Finalmente, podemos graficar las curvas de sintonía promedio para las unidades que prefieren cada uno de los números. ¿Coincide con lo reportado por los autores?

In [ ]:
# Calcular la curva de sintonía promedio de cada unidad
tuning_curves = average_tuning_curves(Q, H, rango_Q)

# Calcular las curvas de sintonía poblacionales para cada numerosidad preferida
tuning_mat = np.array([np.mean(tuning_curves[:,pref_num==q], axis=1) for q in rango_Q])  # una fila por cada numerosidad preferida
tuning_err = np.array([
    np.std(tuning_curves[:,pref_num==q], axis=1) / np.sqrt(np.sum(pref_num==q))          # error estándar para cada punto de cada curva de sintonía
    for q in rango_Q
])

# Normalizar las curvas de sintonía poblacionales al rango 0–1
tmmin = tuning_mat.min(axis=1)[:,None]
tmmax = tuning_mat.max(axis=1)[:,None]
tuning_mat = (tuning_mat - tmmin) / (tmmax - tmmin)
tuning_err = tuning_err / (tmmax - tmmin)   # escalar el error estándar para que sea consistente con la normalización de arriba

# Graficar las curvas de sintonía poblacionales en escala lineal
plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)
for i, (tc, err) in enumerate(zip(tuning_mat, tuning_err)):
    plt.errorbar(rango_Q, tc, err, color=colores_Q[i])
    plt.xticks(rango_Q)
plt.xlabel('Numerosidad')
plt.ylabel('Actividad normalizada')

# Graficar las curvas de sintonía poblacionales en escala logarítmica
plt.subplot(1,2,2)
for i, (tc, err) in enumerate(zip(tuning_mat, tuning_err)):
    plt.errorbar(rango_Q+1, tc, err, color=colores_Q[i])  # desplazar el eje x por uno para evitar tomar logaritmo de cero
    plt.xscale('log', base=2)
    plt.xticks(ticks=rango_Q+1, labels=rango_Q)
plt.xlabel('Numerosidad')
plt.ylabel('Actividad normalizada')
plt.tight_layout()
plt.show()

# Respuestas promedio de las unidades sintonizadas a cero ante numerosidades 1, 2 y 3
R01 = tuning_curves[:,pref_num==0][1]
R02 = tuning_curves[:,pref_num==0][2]
R03 = tuning_curves[:,pref_num==0][3]

## Referencias

Nasr, K., Viswanathan, P. & Nieder, A. (2019). Number detectors spontaneously emerge in a deep neural network designed for visual object recognition. *Science Advances*, 5(5), eaav7903. [doi:10.1126/sciadv.aav7903](https://doi.org/10.1126/sciadv.aav7903)

Kubilius, J., Schrimpf, M., Nayebi, A., Bear, D., Yamins, D. L. K & DiCarlo, J. J. (2018). CORnet: Modeling the Neural Mechanisms of Core Object Recognition. *bioRxiv*, 5(5), eaav7903. [doi:10.1101/408385](https://doi.org/10.1101/408385)